# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 10 2023 2:07PM,259107,10,Eme-110213,Emerginnova,Released
1,Mar 10 2023 2:04PM,259106,10,MSP219175,Methapharm-G,Released
2,Mar 10 2023 2:01PM,259105,10,MSP219308,"Methapharm, Inc.",Released
3,Mar 10 2023 2:01PM,259105,10,MSP219328,"Methapharm, Inc.",Released
4,Mar 10 2023 2:01PM,259104,20,9252955,Else Nutrition,Released
5,Mar 10 2023 1:55PM,259103,22,643459,"NBTY Global, Inc.",Executing
6,Mar 10 2023 1:35PM,259102,19,9252947,VITABIOTICS USA INC.,Released
7,Mar 10 2023 1:35PM,259102,19,9252948,VITABIOTICS USA INC.,Released
8,Mar 10 2023 1:35PM,259102,19,9252950,VITABIOTICS USA INC.,Released
9,Mar 10 2023 1:22PM,259101,10,9243793,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,259103,Executing,1
32,259104,Released,1
33,259105,Released,2
34,259106,Released,1
35,259107,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259103,NaN,1.0,NaN
259104,NaN,NaN,1.0
259105,NaN,NaN,2.0
259106,NaN,NaN,1.0
259107,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259023,0.0,0.0,4.0
259024,0.0,0.0,1.0
259033,0.0,0.0,5.0
259052,0.0,1.0,0.0
259057,9.0,11.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259023,0,0,4
259024,0,0,1
259033,0,0,5
259052,0,1,0
259057,9,11,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259023,0,0,4
1,259024,0,0,1
2,259033,0,0,5
3,259052,0,1,0
4,259057,9,11,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259023,,,4
1,259024,,,1
2,259033,,,5
3,259052,,1,
4,259057,9,11,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 10 2023 2:07PM,259107,10,Emerginnova
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc."
4,Mar 10 2023 2:01PM,259104,20,Else Nutrition
5,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc."
6,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.
9,Mar 10 2023 1:22PM,259101,10,Eywa Pharma Inc.
10,Mar 10 2023 1:04PM,259099,10,Bio-PRF
13,Mar 10 2023 12:49PM,259097,10,"ClearSpec, LLC"
14,Mar 10 2023 12:48PM,259096,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 10 2023 2:07PM,259107,10,Emerginnova,,,1
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G,,,1
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc.",,,2
3,Mar 10 2023 2:01PM,259104,20,Else Nutrition,,,1
4,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc.",,1,
5,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,,,3
6,Mar 10 2023 1:22PM,259101,10,Eywa Pharma Inc.,,,1
7,Mar 10 2023 1:04PM,259099,10,Bio-PRF,,,3
8,Mar 10 2023 12:49PM,259097,10,"ClearSpec, LLC",,,1
9,Mar 10 2023 12:48PM,259096,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 2:07PM,259107,10,Emerginnova,1,,
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1,,
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc.",2,,
3,Mar 10 2023 2:01PM,259104,20,Else Nutrition,1,,
4,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc.",,1,
5,Mar 10 2023 1:35PM,259102,19,VITABIOTICS USA INC.,3,,
6,Mar 10 2023 1:22PM,259101,10,Eywa Pharma Inc.,1,,
7,Mar 10 2023 1:04PM,259099,10,Bio-PRF,3,,
8,Mar 10 2023 12:49PM,259097,10,"ClearSpec, LLC",1,,
9,Mar 10 2023 12:48PM,259096,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 2:07PM,259107,10,Emerginnova,1,,
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1,,
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc.",2,,
3,Mar 10 2023 2:01PM,259104,20,Else Nutrition,1,,
4,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 2:07PM,259107,10,Emerginnova,1.0,NaN,NaN
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1.0,NaN,NaN
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc.",2.0,NaN,NaN
3,Mar 10 2023 2:01PM,259104,20,Else Nutrition,1.0,NaN,NaN
4,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 10 2023 2:07PM,259107,10,Emerginnova,1.0,0.0,0.0
1,Mar 10 2023 2:04PM,259106,10,Methapharm-G,1.0,0.0,0.0
2,Mar 10 2023 2:01PM,259105,10,"Methapharm, Inc.",2.0,0.0,0.0
3,Mar 10 2023 2:01PM,259104,20,Else Nutrition,1.0,0.0,0.0
4,Mar 10 2023 1:55PM,259103,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2850032,33.0,7.0,1.0
12,259091,0.0,1.0,0.0
15,1554311,31.0,0.0,0.0
18,259084,1.0,0.0,0.0
19,1554475,22.0,17.0,11.0
20,259104,1.0,0.0,0.0
21,518123,0.0,2.0,0.0
22,259103,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2850032,33.0,7.0,1.0
1,12,259091,0.0,1.0,0.0
2,15,1554311,31.0,0.0,0.0
3,18,259084,1.0,0.0,0.0
4,19,1554475,22.0,17.0,11.0
5,20,259104,1.0,0.0,0.0
6,21,518123,0.0,2.0,0.0
7,22,259103,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,7.0,1.0
1,12,0.0,1.0,0.0
2,15,31.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,22.0,17.0,11.0
5,20,1.0,0.0,0.0
6,21,0.0,2.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,12,Released,0.0
2,15,Released,31.0
3,18,Released,1.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0
Executing,7.0,1.0,0.0,0.0,17.0,0.0,2.0,1.0
Released,33.0,0.0,31.0,1.0,22.0,1.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,7.0,1.0,0.0,0.0,17.0,0.0,2.0,1.0
2,Released,33.0,0.0,31.0,1.0,22.0,1.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0
1,Executing,7.0,1.0,0.0,0.0,17.0,0.0,2.0,1.0
2,Released,33.0,0.0,31.0,1.0,22.0,1.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()